# Risk

### VaR (Value at Risk)

We cannot eliminate market risk but we can reduce it through various means. The most popular way of accomplishing that is through diversification. Given that our strategy is market-neutral, we should be still protected even in case of a market breakdown. We can measure said risk by using VaR which calculates the maximum potential loss a portfolio could experience within a timeframe.

In [7]:
import polars as pl
import numpy as np
from scipy.stats import norm

df_returns = pl.read_csv('./data/strat/returns.csv')

trade_pnl = df_returns['trade_pnl'].to_numpy()

# initial portfolio value of 1m USD
aum = 1_000_000
mu = np.mean(trade_pnl)
sigma = np.std(trade_pnl)
alpha = norm.ppf(1-0.99, mu, sigma)

print(f"The VaR on a daily basis for our strategy is ${aum-aum*(alpha+1):2.2f} with a 99% confidence")

The VaR on a daily basis for our strategy is $61449.86 with a 99% confidence


### CVaR (Conditional Value at Risk)

Next, we could also explore the conditional value at risk, or expected shortfall which refers to the worst q% of cases over a specific time period. It really is just the tail risk.

$$ES_{\alpha} (X) = -\frac{1}{\alpha} \int_{0}^{\alpha} VaR \gamma (X) d\gamma$$


In [18]:
# order trade pnl, in this case not very interesting
sorted = np.sort(trade_pnl)
cvar_index = int((1-0.99) * len(sorted))

# mean at the cutoff index
cvar = np.mean(sorted[:cvar_index])

print(f"The CVaR for our strategy is {cvar*100:2.2f}% with a 99% confidence")

The CVaR for our strategy is -1.01% with a 99% confidence


### Maximum Drawdown

Given the limited data set and, consequently, the number of trades, the maximum drawdown of the strategy is somewhat limited. In our case, we had one losing trade and the maximum drawdown was -15.7% which completely recovered in February 2024, or after 3 months.

In [39]:
df_returns = df_returns.with_columns(
    roll_max = pl.col('cumulative_pnl').cum_max()
).with_columns(
    drawdown = pl.col('cumulative_pnl')/pl.col('roll_max') - 1.0
)

df_returns[['cumulative_pnl', 'drawdown']].plot()

descr,bitcoin-cash,dogecoin,z,x_dif,y_dif,trade_pnl,date,cumulative_pnl,drawdown,roll_max
str,f64,f64,f64,f64,f64,f64,str,f64,f64,f64
"""close short""",238.503769,0.071241,-0.235986,0.198633,0.168271,-0.030363,"""2023-11-06""",0.163187,-0.156873,0.19355
null,0.0,0.0,0.0,0.0,0.0,0.0,"""2023-11-07""",0.163187,-0.156873,0.19355
"""long""",246.020996,0.075554,-0.516764,0.031518,0.060551,0.0,"""2023-11-08""",0.163187,-0.156873,0.19355
null,0.0,0.0,0.0,0.0,0.0,0.0,"""2023-11-09""",0.163187,-0.156873,0.19355
null,0.0,0.0,0.0,0.0,0.0,0.0,"""2023-11-10""",0.163187,-0.156873,0.19355
…,…,…,…,…,…,…,…,…,…,…
null,0.0,0.0,0.0,0.0,0.0,0.0,"""2024-02-07""",0.16605,-0.14208,0.19355
null,0.0,0.0,0.0,0.0,0.0,0.0,"""2024-02-08""",0.16605,-0.14208,0.19355
null,0.0,0.0,0.0,0.0,0.0,0.0,"""2024-02-09""",0.16605,-0.14208,0.19355
